In [5]:
import pandas as pd
import numpy as np
import json
from os import listdir
from os.path import isfile, join

In [6]:
def load_files(filedir):
  with open(filedir) as f:
    file = json.loads(f.read())
    return file

# Load Files

!git clone https://github.com/marcoramilli/MalwareTrainingSets/

In [7]:
mypath='/Users/fernanda/Documents/Hands On Machine Learning for Cybersecurity/Cybersecurity-MachineLearning/MalwareTrainingSets/trainingSets'
malwaretype=["APT1","Crypto","Locker","Zeus"]

apt1_files=[join(join(mypath,malwaretype[0]),f) for f in listdir(join(mypath,malwaretype[0])) if isfile(join(join(mypath,malwaretype[0]), f))]
crypto_files=[join(join(mypath,malwaretype[1]),f) for f in listdir(join(mypath,malwaretype[1])) if isfile(join(join(mypath,malwaretype[1]), f))]
locker_files=[join(join(mypath,malwaretype[2]),f) for f in listdir(join(mypath,malwaretype[2])) if isfile(join(join(mypath,malwaretype[2]), f))]
zeus_files=[join(join(mypath,malwaretype[3]),f) for f in listdir(join(mypath,malwaretype[3])) if isfile(join(join(mypath,malwaretype[3]), f))]

apt1_files = [load_files(filedir=name) for name in apt1_files]
crypto_files = [load_files(filedir=name) for name in crypto_files]
locker_files = [load_files(filedir=name) for name in locker_files]
zeus_files = [load_files(filedir=name) for name in zeus_files]

In [27]:
# archive structure:
print(apt1_files[0].keys())
print(crypto_files[0].keys())
print(locker_files[0].keys())
print(zeus_files[0].keys())
print("\n")
print('properties apt1 file 0:',len(apt1_files[0]['properties'].keys()))
print('properties crypto file 0:',len(crypto_files[0]['properties'].keys()))
print('properties locker file 0:',len(locker_files[0]['properties'].keys()))
print('properties zeus file 0:',len(zeus_files[0]['properties'].keys()))
#each kind of malware has different properties in its files

dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])
dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])
dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])
dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])


properties apt1 file 0: 22
properties crypto file 0: 18
properties locker file 0: 17
properties zeus file 0: 34


In [28]:
apt1_prop = []
crypto_prop = []
locker_prop = []
zeus_prop = []

for file in apt1_files:
  for prop in file['properties']:
    if (prop not in apt1_prop):
        apt1_prop.append(prop)
for file in crypto_files:
  for prop in file['properties']:
    if (prop not in crypto_prop):
        crypto_prop.append(prop)
for file in locker_files:
  for prop in file['properties']:
    if (prop not in locker_prop):
        locker_prop.append(prop)
for file in zeus_files:
  for prop in file['properties']:
    if (prop not in zeus_prop):
        zeus_prop.append(prop)

#check common properties
props = []
for prop in apt1_prop:
  if (prop in crypto_prop and prop in locker_prop and zeus_prop):
    props.append(prop)

print('common properties:',len(props))

common properties: 52


In [49]:
## create dataset
df=pd.DataFrame(columns=props)
# for f in apt1_files:
#     for i,prop in enumerate(props):
        
df[0].append(apt1_files[0]['properties'].get(props[0]))

KeyError: 0